# Importation de packages

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from rich.table import Table
from rich import print
import pandas as pd
import numpy as np
import pickle

# Importation de la base de données

In [2]:
df = pd.read_csv('data.csv', sep = ';')
df.reset_index(drop = True).head()

,Date,Team_Home,Team_Visitor,Points_Visitor,Field_Goals_Made_V,Field_Goals_Attempted_V,Field_Goal_Percentage_V,Three_Point_Field_Goals_Made_V,Three_Point_Field_Goals_Attempted_V,Three_Point_Field_Goal_Percentage_V,...,Blocks_H,Turnovers_H,Personal_Fouls_H,Season,Home_win,Visitor_win,Team_Home_Elo_Before,Team_Visitor_Elo_Before,Team_Home_Elo_After,Team_Visitor_Elo_After
0,2006-11-03,Indiana Pacers,New Orleans/Oklahoma City Hornets,91.0,36.0,84.0,0.429,1.0,11.0,0.091,...,5.0,17.0,26.0,2006-2007,0,1,1510.201113,1507.668775,1498.306377,1519.563512
1,2006-11-03,Orlando Magic,Philadelphia 76ers,88.0,31.0,74.0,0.419,3.0,9.0,0.333,...,7.0,23.0,23.0,2006-2007,0,1,1510.409919,1508.486104,1504.516331,1514.379692
2,2006-11-03,Toronto Raptors,Milwaukee Bucks,105.0,43.0,93.0,0.462,2.0,14.0,0.143,...,5.0,15.0,27.0,2006-2007,1,0,1491.828491,1511.009351,1504.334639,1498.503203
3,2006-11-03,Atlanta Hawks,New York Knicks,118.0,40.0,95.0,0.421,10.0,22.0,0.455,...,4.0,18.0,24.0,2006-2007,1,0,1490.351861,1504.901123,1499.042942,1496.210042
4,2006-11-03,Boston Celtics,Detroit Pistons,97.0,31.0,72.0,0.431,8.0,21.0,0.381,...,6.0,19.0,31.0,2006-2007,0,1,1495.498419,1492.992987,1480.526750,1507.964655


# Échantillonnage

In [3]:
df_train = df.drop(columns = ["Date", "Team_Home", "Team_Visitor", 'Season', "Visitor_win", "Team_Home_Elo_After", "Team_Visitor_Elo_After"])
features = df_train.drop(columns = 'Home_win')
target = df_train['Home_win']
features.head()

,Points_Visitor,Field_Goals_Made_V,Field_Goals_Attempted_V,Field_Goal_Percentage_V,Three_Point_Field_Goals_Made_V,Three_Point_Field_Goals_Attempted_V,Three_Point_Field_Goal_Percentage_V,Free_Throws_Made_V,Free_Throws_Attempted_V,Free_Throw_Percentage_V,...,Offensive_Rebounds_H,Defensive_Rebounds_H,Rebounds_H,Assists_H,Steals_H,Blocks_H,Turnovers_H,Personal_Fouls_H,Team_Home_Elo_Before,Team_Visitor_Elo_Before
0,91.0,36.0,84.0,0.429,1.0,11.0,0.091,18.0,33.0,0.545,...,18.0,32.0,50.0,26.0,14.0,5.0,17.0,26.0,1510.201113,1507.668775
1,88.0,31.0,74.0,0.419,3.0,9.0,0.333,23.0,25.0,0.920,...,6.0,30.0,36.0,21.0,7.0,7.0,23.0,23.0,1510.409919,1508.486104
2,105.0,43.0,93.0,0.462,2.0,14.0,0.143,17.0,25.0,0.680,...,7.0,31.0,38.0,21.0,8.0,5.0,15.0,27.0,1491.828491,1511.009351
3,118.0,40.0,95.0,0.421,10.0,22.0,0.455,28.0,36.0,0.778,...,17.0,25.0,42.0,11.0,9.0,4.0,18.0,24.0,1490.351861,1504.901123
4,97.0,31.0,72.0,0.431,8.0,21.0,0.381,27.0,34.0,0.794,...,10.0,42.0,52.0,18.0,5.0,6.0,19.0,31.0,1495.498419,1492.992987


In [37]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state=42)

# Les modèles

### 1. Logistic regression

In [42]:
logistic_grid = {
    'penalty': ['l2', 'penalty', 'none'], 
    'solver': ['sag'],
    'max_iter': [int(x) for x in np.linspace(start = 0, stop = 500, num = 15)],
    'multi_class': ['auto', 'multinomial'], 
    'C': [0.5, 1.0, 1.5, 10]
}

model_logistic = GridSearchCV(estimator = LogisticRegression(), param_grid = logistic_grid, cv = 5, n_jobs = -1)
model_logistic.fit(x_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1113: UserWarning: Setting penalty='none' will ignore the

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.5, 1.0, 1.5, 10],
                         'max_iter': [0, 35, 71, 107, 142, 178, 214, 250, 285,
                                      321, 357, 392, 428, 464, 500],
                         'multi_class': ['auto', 'multinomial'],
                         'penalty': ['l2', 'penalty', 'none'],
                         'solver': ['sag']})

In [43]:
best_logistic = model_logistic.best_estimator_
best_logistic.fit(x_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=178, multi_class='multinomial', penalty='none',
                   solver='sag')

In [44]:
pickle.dump(best_logistic, open('logistic_regression_model.sav', 'wb'))

In [45]:
train_score_logistic = best_logistic.score(x_train,y_train)
test_score_logistic = best_logistic.score(x_test,y_test)
train_score_logistic

0.6458763596310064

### 2. Random Forest Classifier

In [46]:
max_depth = [int(x) for x in np.linspace(50, 100, num = 3)]
max_depth.append(None)

def multiply_int_by_two(x, max):
    liste_int = []
    while True :
        x = x*2
        if x < max:
            liste_int.append(x)
        else: 
            break
    return liste_int

random_grid = {'n_estimators': multiply_int_by_two(4, 300),
               'max_features': ['auto'],
               'max_depth': max_depth,
               'bootstrap': [True]
            }
            
model_random = GridSearchCV(estimator = RandomForestClassifier(), param_grid = random_grid, cv = 5, n_jobs = -1)
model_random.fit(x_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForest

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [50, 75, 100, None],
                         'max_features': ['auto'],
                         'n_estimators': [8, 16, 32, 64, 128, 256]})

In [47]:
best_random = model_random.best_estimator_
best_random.fit(x_train, y_train)

/usr/local/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_features='auto', n_estimators=256)

In [48]:
pickle.dump(best_random, open('random_forest_model.sav', 'wb'))

In [49]:
train_score_random = best_random.score(x_train,y_train)
test_score_random = best_random.score(x_test,y_test)
train_score_random

1.0

### 3. Ridge Classifier

In [50]:
ridge_grid = {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0],
            'fit_intercept': [True, False]
            }

model_ridge = GridSearchCV(estimator = RidgeClassifier(), param_grid = ridge_grid, cv = 5, n_jobs = -1)
model_ridge.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RidgeClassifier(), n_jobs=-1,
             param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0],
                         'fit_intercept': [True, False]})

In [51]:
best_ridge = model_ridge.best_estimator_
best_ridge.fit(x_train, y_train)

RidgeClassifier(alpha=0.001)

In [52]:
pickle.dump(best_ridge, open('ridge_classifier_model.sav', 'wb'))

In [53]:
train_score_ridge = best_ridge.score(x_train,y_train)
test_score_ridge = best_ridge.score(x_test,y_test)
train_score_ridge

0.647803937766763

### 4. Neural Network

In [54]:
pipeline_neural = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neural", MLPRegressor()),
    ]
)

neural_grid = {
  'neural__alpha': 10.0 ** -np.arange(1, 7),
  'neural__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
  'neural__activation': ['logistic', 'relu'],
  'neural__batch_size': multiply_int_by_two(8, 300)
}

model_neural = GridSearchCV(estimator = pipeline_neural, param_grid = neural_grid, cv = 5, n_jobs = -1)
model_neural.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('mise_echelle', MinMaxScaler()),
                                       ('neural', MLPRegressor())]),
             n_jobs=-1,
             param_grid={'neural__activation': ['logistic', 'relu'],
                         'neural__alpha': array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06]),
                         'neural__batch_size': [16, 32, 64, 128, 256],
                         'neural__hidden_layer_sizes': ((25,), (50,), (100,),
                                                        (20, 20))})

In [55]:
best_neural = model_neural.best_estimator_
best_neural.fit(x_train, y_train)

Pipeline(steps=[('mise_echelle', MinMaxScaler()),
                ('neural', MLPRegressor(alpha=0.1, batch_size=256))])

In [56]:
pickle.dump(best_neural, open('neural_network_model.sav', 'wb'))

In [57]:
train_score_neural = best_neural.score(x_train,y_train)
test_score_neural = best_neural.score(x_test,y_test)
train_score_neural

0.09824128816263511

### 5. Support Vector Classification

In [58]:
pipeline_svc = Pipeline(
    [
        ("anova", SelectPercentile(chi2)),
        ("mise_echelle", MinMaxScaler()),
        ("svc", SVC()),
    ]
)

svc_grid = {
  'svc__C': [0.1, 1.0, 10], 
  'svc__gamma': [1.0, 0.1, 0.01, 0.001],
  'svc__kernel': ['rbf', 'poly', 'sigmoid']
}

model_svc = GridSearchCV(estimator = pipeline_svc, param_grid = svc_grid, cv = 5, n_jobs = -1)
model_svc.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('anova',
                                        SelectPercentile(score_func=<function chi2 at 0x13109b550>)),
                                       ('mise_echelle', MinMaxScaler()),
                                       ('svc', SVC())]),
             n_jobs=-1,
             param_grid={'svc__C': [0.1, 1.0, 10],
                         'svc__gamma': [1.0, 0.1, 0.01, 0.001],
                         'svc__kernel': ['rbf', 'poly', 'sigmoid']})

In [59]:
best_svc = model_svc.best_estimator_
best_svc.fit(x_train, y_train)

Pipeline(steps=[('anova',
                 SelectPercentile(score_func=<function chi2 at 0x13109b550>)),
                ('mise_echelle', MinMaxScaler()),
                ('svc', SVC(C=10, gamma=0.1, kernel='sigmoid'))])

In [60]:
pickle.dump(best_svc, open('svc_model.sav', 'wb'))

In [61]:
train_score_svc = best_svc.score(x_train,y_train)
test_score_svc = best_svc.score(x_test,y_test)
train_score_svc

0.6438110973426958

### 6. Naive Bayes

In [62]:
naives_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
}

model_naives = GridSearchCV(estimator = GaussianNB(), param_grid = naives_grid, cv = 5, n_jobs = -1)
model_naives.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05,
                                           0.0001, 0.001, 0.01, 0.1]})

In [63]:
best_naives = model_naives.best_estimator_
best_naives.fit(x_train, y_train)

GaussianNB(var_smoothing=0.1)

In [64]:
pickle.dump(best_naives, open('naives_bayes_model.sav', 'wb'))

In [65]:
train_score_naives = best_naives.score(x_train,y_train)
test_score_naives = best_naives.score(x_test,y_test)
train_score_naives

0.640850888062784

### 7. K-Nearest Neighbors Classifier

In [66]:
pipeline_knn = Pipeline(
    [
        ("anova", SelectPercentile(chi2)),
        ("mise_echelle", MinMaxScaler()),
        ("knn", KNeighborsClassifier()),
    ]
)

knn_grid = {
    'knn__n_neighbors': range(5, 15),
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

model_knn = GridSearchCV(estimator = pipeline_knn, param_grid = knn_grid, cv = 5, n_jobs = -1)
model_knn.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('anova',
                                        SelectPercentile(score_func=<function chi2 at 0x13109b550>)),
                                       ('mise_echelle', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': range(5, 15),
                         'knn__weights': ['uniform', 'distance']})

In [67]:
best_knn = model_knn.best_estimator_
best_knn.fit(x_train, y_train)

Pipeline(steps=[('anova',
                 SelectPercentile(score_func=<function chi2 at 0x13109b550>)),
                ('mise_echelle', MinMaxScaler()),
                ('knn',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=13))])

In [68]:
pickle.dump(best_knn, open('knn_model.sav', 'wb'))

In [69]:
train_score_knn = best_knn.score(x_train,y_train)
test_score_knn = best_knn.score(x_test,y_test)
train_score_knn

0.677956767176098

# Comparaison des modèles

In [70]:
def suppression_model_overfitting(dict_score_models: dict[str, list[float]]) -> dict[str, list[float]]:
    """Supprime du dictionnaire les modèles en surapprentissage."""
    liste = []
    for model in dict_score_models:
        if abs(dict_score_models[model][0] - dict_score_models[model][1]) > 0.15:
            liste.append(model)
        else:
            pass

    [dict_score_models.pop(x, None) for x in liste]
    return dict_score_models

In [71]:
def selection_best_model(n: float) -> tuple[str, float]:
    """Sélectionne le meilleur modèle, selon le score de cross validation, parmis tous les modèles."""
    score_models = {
        "Logistic Regression": [train_score_logistic, test_score_logistic],
        "Ridge Classifier": [train_score_ridge, test_score_ridge],
        "RandomForestClassifier": [train_score_random, test_score_random],
        "Neural Network": [train_score_neural, test_score_neural],
        "Support Vector Classification": [train_score_svc, test_score_svc],
        "Naive Bayes": [train_score_naives, test_score_naives],
        "K-Nearest Neighbors Classifier": [train_score_knn, test_score_knn],
    }
    score_models = suppression_model_overfitting(score_models)
    best_model = max(score_models, key=score_models.get)
    best_score_cross_validation = score_models[best_model][0]
    return best_model, best_score_cross_validation

In [72]:
def affichage(n: float) -> str:
    """Fonction qui permet d'afficher le résultat."""
    best_model = selection_best_model(n)
    model = best_model[0]
    score = round(best_model[1], 5)
    print(
        f"Le meilleur modèle est {model}, avec un score de cross_validation de {score}.")

In [73]:
affichage(0.15)

Le meilleur modèle est K-Nearest Neighbors Classifier, avec un score de cross_validation de 0.67796.